In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/grocery-item-feature-extraction/README.dataset.txt
/kaggle/input/grocery-item-feature-extraction/README.roboflow.txt
/kaggle/input/grocery-item-feature-extraction/data.yaml
/kaggle/input/grocery-item-feature-extraction/valid/labels/image_256_jpg.rf.16b4577fd2d07058c44e37ad56e714a8.txt
/kaggle/input/grocery-item-feature-extraction/valid/labels/20230310_110956_jpg.rf.3de9c80ab3a6ea76f1f7a4004e036086.txt
/kaggle/input/grocery-item-feature-extraction/valid/labels/image_43_jpg.rf.70a6b53f6c50030523f6541f0d66bbb9.txt
/kaggle/input/grocery-item-feature-extraction/valid/labels/20230730_110324_jpg.rf.7be23cf457d1b21ee27d749b69270572.txt
/kaggle/input/grocery-item-feature-extraction/valid/labels/image_59_jpg.rf.b3197866f891ac16e3f257ff60085d8a.txt
/kaggle/input/grocery-item-feature-extraction/valid/labels/image_163_jpg.rf.21b33cc91f2af3de7ece5e0627ffdbcf.txt
/kaggle/input/grocery-item-feature-extraction/valid/labels/20230310_110901_jpg.rf.3c068da0cfeb236f5a3e8c123c8470c0.txt
/kaggl

In [2]:
# 1. Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import os
import cv2
from tqdm import tqdm
import shutil
from pathlib import Path

In [3]:
# 2. Configure GPU and memory settings
try:
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("GPU is available and configured")
    else:
        print("No GPU devices found")
except:
    print("Error in GPU configuration")

GPU is available and configured


In [4]:
# 3. Set up directory structure
def setup_directories():
    """
    Create necessary directory structure for the model
    """
    # Base directories
    DATASET_PATH = '/kaggle/input/grocery-item-feature-extraction'
    OUTPUT_PATH = '/kaggle/working/grocery_classification'
    
    # Create directories
    directories = {
        'train': os.path.join(OUTPUT_PATH, 'train'),
        'valid': os.path.join(OUTPUT_PATH, 'valid'),
        'test': os.path.join(OUTPUT_PATH, 'test'),
        'models': os.path.join(OUTPUT_PATH, 'models'),
        'logs': os.path.join(OUTPUT_PATH, 'logs')
    }
    
    for dir_path in directories.values():
        os.makedirs(dir_path, exist_ok=True)
    
    return directories

# Create class mapping
def create_class_mapping():
    """
    Create mapping between class names and indices
    """
    class_names = [
    "ALphonsoMangoPulp_GoldenCrown", "Aakash", "Aarambh", "Aashirvaad", "Aashirvaad_Besan",
    "Aashirvaad_TurmericPowder", "Abzorb", "AcaciaHoney_KhushNuma", "Action_Dishwash_Liquid",
    "ActiveSaltNeem_Toothpaste_Colgate", "ActiveSalt_Toothpaste_Colgate", "Agarbathi_Bansuri",
    "Agarbathi_Cycle", "Agarbatti_SalShakti", "Agastya", "Agrawal", "AirFreshner", "AllOut",
    "Almendro", "AlmondFlour_Granos", "AlmondKernals_Aram-s", "Almonds", "AloFrut_Juice",
    "Aloevera_Handwash_Patanjali", "Aloo Bhujia", "AlooBhujia_Bicano", "AlpineFresh_BodySpray_ShotLayerr",
    "AlterEgo_BodySpray_ParkAvenue", "AmericanGarden", "AmericanPancakeCo", "AmlaCandy_Patanjali",
    "AmlaOil", "Amul", "Amul Butter", "Amul ghee", "Amul-Dreamlite_Biscuites", "AmulCool",
    "Amul_BadamShake", "Amul_ButterCookies", "Amul_Kool", "Amulya", "AntiRoachgel_Hit", "Anymany",
    "Apis", "AppleCiderVinegar_AmericanGarden", "AppleCiderVinegar_CanadianGarden",
    "AppleCiderVinegar_JustOrganic", "AppleCiderVinegar_WOW", "AppleCiderVinegar_Wanposh",
    "Apple_Juice_Tropicana", "AppyFizz", "Apricots", "Apsara", "Aravalli_Rice", "Ariel", "Ashok",
    "BC", "BLackCode_Denver", "BNatural", "Babe_BodySpray_Vanesa", "BabyCereal", "BabyCorn_GoldenCrown",
    "BabyWash_Himalaya", "BadamMilkshake_Cavins", "Baggrys", "Bagrrys", "Baidyanath", "Bajaj",
    "Bakeshree", "BakingPaper", "BakingPowder", "Balaji", "Bambino", "Banana_chips", "BanneNawabs",
    "BasilSeed_Ajwah", "BasilSeeds", "Batchelors", "BathingSoap_LifeBuoy", "BathingSoap_Pears",
    "Bathroom_Cleaner", "Bauli", "BauliSavoriz", "Beans", "Belgian", "BerryJam_Mala",
    "BesanLaddoo_BigBasket", "Besan_Rajdhani", "Bhujia_Bicano", "Bhujia_Haldirams", "Bhujiya_Prabhuji",
    "Bicano_PanjeeriLaddu", "Bigbabol_Chewinggum", "Bikaji", "Bikaji Tana bana", "Bikaji_navratan mixtute",
    "Bikano", "Bikano Aloo Bhujia", "Bikano Royal Rasgulla", "Bikano Tasty", "BikanoRasgulla", "Bingo",
    "BiotinTablets_HKVitals", "Biotique", "Biscuit", "Biscuit_Bourbon_Britania",
    "Biscuit_ButterBite_PriyaGold", "Biscuit_ChocoRolls_HideandSeek", "Biscuit_DarkFantasy",
    "Biscuit_Digestive_FirstCrop", "Biscuits_Amul", "Bisleri", "BlackMagic_Deo_Spinz",
    "BlackPepperPowder_MDH", "BlackPepper_Catch", "Bleach_Fem", "BlueBird", "BlueCurrents_St.John",
    "Blueberries", "Blueberry_Gaurmia", "Blush_BodySpray_Eva", "Blush_FragrantTalc_Envy",
    "BodyHairRemover_Veet", "BodyLotion_Parachute", "BodyMist_Ossum", "BodySpray_BoldSecret",
    "BodySpray_Eva", "BodySpray_Jovan", "BodySpray_ParkAvenue", "BodySpray_The_Man_Company",
    "BodySpray_Women", "BodySpray_men", "BombayBiryani_Alamin", "Bookside", "Boondi_Jabson-s",
    "Boost", "Borges", "BoroPlus", "Boroline", "Bounce", "Bounty", "Bourbon cream biscuits",
    "Bread", "Brij_Bhujia", "Brij_Namkeen", "Brij_Panchratan", "Britannia",
    "Britannia_WinkinCow_lassi", "BrookeBond", "Brown-Haley", "BrownRice", "BrowniesBasket",
    "Bru", "BubbleGumFlovour_ToothpasteForKids_DentoShine", "Budweiser", "Buldak_Ramen",
    "Bumtum_BabyPaints", "Bumtum_DoobiDoo", "Butlers", "Butter", "ButterBatter", "ButterBite_Cookies",
    "ButterCookies_Amul", "ButterCookies_Unibic", "Butter_AmulLite", "Butter_MilkyMist",
    "Butter_MotherDairy", "Butter_Nutralite", "ButtonMushroom_GoldenCrown", "Cadbury",
    "Cadbury_Celebrations", "Cake", "Candy", "Candy_Butter_PureBurst", "Candy_Halls",
    "Candy_KachchaAam_PureBurst", "Candy_Lacmi", "Candy_Mandola", "Candy_Mentos",
    "Candy_Orange_PureBurst", "Candy_PureBurst_KachchaAam", "Candy_PureBurst_MangoTango",
    "Candy_PureBurst_Orange", "Candy_PureBurst_chocolate", "Candyman_ChocoDouble",
    "CaramelBliss_Popcorn_ActII", "CashewSticks_GoneMad", "Cashews", "CastorOil", "Catch",
    "Catch Black Pepper", "Catch Table Salt", "Catch chicken masala", "Catch_GaramMasala",
    "Catch_KashmiriMirchi", "Catch_MeatMasala", "Catch_jeera powder", "Cavin",
    "Cavin_VaneelaMilkShake", "Cavins", "Cerelac_Apple_Nestle", "Cerelac_Wheat_Nestle",
    "ChakkiAtta", "ChanaMasala_Jobsons", "ChanaMasala_MinuteMeals_MTR", "ChanaMasala_MinuteMeals_Mir",
    "CharcoalCleangel_Toothpaste_Colgate", "Cheese", "CheeseCubes", "CheeseCubes_D-lecta",
    "CheeseCubes_MilkyMist", "CheeseCubes_MotherDairy", "CheeseSlices", "CheeseSlices_Amul",
    "CheeseSlices_Britannia", "CheeseSlices_D-lecta", "CheeseSlices_Go", "CheeseSlices_MilkyMist",
    "CheeseSlices_MotherDairy", "Cheese_D-lecta", "Cheetos", "Cheezza_Britannia", "ChefsBasket",
    "ChiaSeed_Ajwah", "ChiaSeeds", "ChickenMasala_Orika", "Chik", "ChileanInshellWalnut_Happilo",
    "ChileanInshellWalnut_nutraj", "ChileanInshellcashenut_Happilo", "ChileanInshellcashenut_nutraj",
    "ChilliPowder_Kanwal", "Chings", "Chips_Bicano", "Chips_Crisps_Lays", "Chips_Doritos",
    "Chips_Haldirams", "ChocoBakes_Cookies_Cadbury", "ChocoFills_Kellogg-s",
    "ChocoHazlenut_Cookies_Unibic", "ChocoNutCookies_Unibic", "ChocoPie", "Chocoflakes_Kwality",
    "Chocola", "ChocolairsGoldCoffee_Cadbury", "Chocolate", "Chocolate_ChocoBarXL",
    "Chocolate_DairyMilk_Cadbury", "Chocolate_Horlicks", "Chocolate_Nutties_Cadburry",
    "Chocolate_Snakker_PriyaGold", "Chocolate_Truffel_Dukes", "Chocozay", "ChoiceMega",
    "Chokolas", "Cibaca_Toothpaste_Colgate", "ClassicButter_Unibic", "ClassicMalt_Horlicks",
    "ClassicSalted_Popcorn_ActII", "Classicgentleman_Perfume_Yardley", "Clean-Clear",
    "ClinicPlus", "Cockroach_repllent_baygon", "Cocktail", "Cocoa_Cadbury", "Cocoa_Hershey-s",
    "CoconutCookies_Americana", "CoconutLadoo_Ajwah", "CoconutOil_Nirmal", "CoconutPeda_Ajwah",
    "CoconutPowder_Sunrise", "CoffeeMate_Nestle", "Coffee_FullBloom", "ColaCandy_Ajwah",
    "Colgate", "Colgate_BubbleFruit_ChihldPaste", "Colgate_Strawberry_ChihldPaste",
    "Colgate_Toothbrush", "Colin", "Comfort", "Complan", "CookTop", "Cookies",
    "CoolKick_NiveaMen", "CoolWave_Talc_Envy", "Corainder_Everest", "CornFlakes_FirstCrop",
    "CornStarch_Brown-Polson", "Cornflour", "Cornitos", "CoughDrops_Honitus_Dabur",
    "CoughDrops_Vicks", "Crackers_Munchy-s", "Cranberrries_Granos", "Cream_Lakme",
    "Cream_vitaminFaceWash", "CremesBiscuit_Strawberry_Cremica", "CrunchyChocolate_Pepero",
    "CrunchyMunchy_Bicano", "CupCakes", "Curd", "Curd_MilkyMist", "Curry_TopRamen",
    "CustardPowder_Brown-Polson", "Czar_Scent_Fogg", "DLecta", "Daawat", "Daawat_RozanaBasmatiRiceGold", "Dabur", "DaimSnax", "DairyMilk_Chocolate",
    "DairyMilk_Silk_Cadbury", "DalMakhani_RedayMeals_Gits", "Dal_Bikano", "Dal_KhattaMeetha_BC",
    "Dal_Rajdhani", "Dant Rakshak", "DantKanti_Toothpaste_Patanjali", "DarkChocolateChipsCookies_Gaia",
    "DarkFantasy_ChocolateShake", "DarkTemptation_CologneTalc_Axe", "DarkTemptation_Fragrance_Axe",
    "Dates", "DatesSyrup_Lion", "Davidoff", "DelMonte", "DelMonte_Mayonnaise", "Delfi", "Delicious",
    "Delifresh", "Delight", "Delisious", "Delmonte_Pineapple", "Delmonte_peachjuice", "Denver",
    "DetergentBar_Henko", "Dettol", "Dev", "Dexolac", "Dhara", "Dhara_Oil", "DiSano",
    "DiabedicCare_Nimbark", "Diamond", "Diapers", "Diapers_MamyPokoPants", "Diapers_Pampers",
    "Discover_BodySpray_ParkAvenue", "Dishwash_Prill", "Dishwash_Scrubz", "Dishwasher_Exo",
    "Diva_BodySpray_Vanesa", "Divine Agro Kasuri Methi", "Divya", "DogFood_Pedigree", "Domex",
    "Don_PerfumeDoeSpray_Beardo", "Donut_Ziggy", "DoobiDoo_BabyPaints", "Doritos", "Dorje", "Dove",
    "Dove_FacialTissue", "DrOetker", "DryFruit", "DryFruitPanjeeriLadoo_Haldiram", "DryFruits",
    "Dukes", "Dyna Sandal", "EasyFun", "EclairOs", "Eclairs", "Eclairs_PureBurst", "EcoValley",
    "Eggs", "Elite", "Emami", "Emperia", "Enchante_Spinz", "EnergyMax", "Engage", "Eno", "Ensure",
    "Enzo", "Epigamia", "Everest", "Everyuth", "Exotic_Spinz", "Ezee", "FabricConditioner_Softlan",
    "Fair-Lovely", "FairMart", "Fairmart", "FaraliChiwda_Bhujialalji", "FastCard_GoodNight",
    "FennelSeedPowder_Kanwal", "FerreroRocher", "Fiama", "FieryPride_BodySpray_ShotLayerr",
    "Fiesta_Dal", "Figaro", "Figs", "Finosta", "First Crop Bikaneri Bhuji", "First Crop Potato chips",
    "First crop Corn Flakes", "FishOil_HKVitals", "Flakes_Choco_FirstCrop", "FlaxSeeds_Nutraj",
    "FlossPicks_DeepClean_DentoShine", "FlossPicks_DentoShine", "Fogg", "FoodCoast",
    "ForestSpice_Deodrant_WildStone", "Fortune", "Fortune besen", "Fortune chana sattu",
    "Fortune suji", "Forture indori poha", "Fox-s", "Foxs", "FreshComfort_Deodrant_Nivea",
    "FreshGel_Sensodyne", "FreshMint_Sensodyne", "Fresho", "FrozenPackagedFood",
    "Fruit-Nut_Cookies_Unibic", "FruitCocktail_GoldenCrown", "FruitGummies_Tapi", "Fruitins",
    "Frutins", "Fulvadi_Jabson", "GM", "GMFoods", "Gaia", "GaiaLite", "Gaialite", "Galaxy",
    "Gamnuts_DryFruits", "Gamnuts_Masale", "Gangwal", "GaramMasala_Orika",
    "Garden Bhelpuri - Sevpuri", "GarlicPaste_Nilon-s", "Garnier", "Gasona_Kudos", "Geham",
    "Geisha", "Genteel", "Gentleman_Perfume_YardleyLondon", "Gerber", "GetReal", "Ghadi", "Ghee",
    "Gillette", "Girnar", "Gits", "Glicy", "Glow", "Glow-Handsome", "Glow-Lovely",
    "GlucoPlusC_Dabur", "GlucoPlusD_Dabur", "GluconD", "Glycerin", "Go", "GoalSportingClub_Denver",
    "Godrej", "Gold_Nescafe", "GoldenScent_Next", "GoneMad", "Good Life toor dal", "Good life",
    "Good life masoor", "Good life urad whole", "GoodDay", "GoodHome", "GoodLife", "GoodMorning",
    "Gourmet-sDelite", "Gowardhan", "Grand_Coffee_Tata", "GreenChilliPickle_Tops",
    "GreenTea_Pocky", "GupShupPeanuts_Haldirams", "Guruji", "Gustora", "Guylian", "Hair-Care",
    "HairColor", "HairOil_Clear", "HairOil_Indulekha", "Hair_Color_ColorMate", "Hajmola",
    "Haldiram-s_Chips", "Haldirams", "Haldirams_KajuMixture", "Haldirams_MoongDal",
    "Haldirams_Peanuts", "Hamilton_Perfume_Denver", "HandSanitizer", "HandSanitizer_Dettol",
    "HandSanitizer_Himalaya", "HandSanitizer_Lifeboy", "HandwashVitamins_LifeBouy",
    "Handwash_Herbal_Patanjali", "Handwash_Santoor", "Happilo", "Happilo_PeanutButter",
    "Haribo", "Harpic", "HastyTasty", "Hawanan Barbeque Cheese Popcorn", "Head-Shoulders",
    "HealthyLife", "Heinz", "HellMann-s_Mayonnaise", "Hellmanns", "Hem", "Herbal_Toothpaste_Colgate",
    "Hersheys", "Hersheys_MilkShake_vaneeliFlavour", "Hide-Seek_Chocolate_Parle",
    "Hide-Seek_CremeSandwiches", "Hide-Seek_Milano_ParlePlatina", "Himalaya", "Hing_Ramdev",
    "Hit", "Hitkary", "Hocco", "HoneyOatmeal_Cookies_Unibic", "Honey_Barosi", "Honey_GoldDrops",
    "Honey_Lion", "Horlicks", "Horlicks women-s plus", "Hurricane_EnergyDrink",
    "HyderabadiBiryani_Rehmat", "HydraEnergy_Deodrant_WildStone",  "IceCream", "IceCream_Amul", "IceCream_BaskinRobbins", "ImliCandy_Ajwah", "IndiShop",
    "IndianSweet_MaysorePak_Nandini", "Indulekha", "Inhaler_Vicks", "Insect_Killer_Hit",
    "Insect_Killer_strategi", "Insight_Perfume", "Intense_Fragrance_Axe", "Iodex", "JMRFoods",
    "Jabsons", "Jackpot", "JaggeryPowder", "JainsTrupti", "Jalani", "Jaljira", "Jam_Kissan",
    "Jam_MixedFruit_Kissan", "Jam_Patanjali", "Jel_SetWet", "Jet_Imli_Toffee", "JewelFarmer",
    "Johnsons", "Joiner_Drink", "Joshina_Hamdard", "Jovees", "Joy", "Jugnu_NapthaleneBalls",
    "Juice_Swing", "Juice_coconut_Paper_Boat", "JuniorTomatoKetchup_Tops", "Kaffe",
    "KalongiSeeds_Ajwah", "Kamasutra", "Kapiva", "KasundiSauce_Elmac", "Kehwa_Aram-s",
    "Kelloggs Corn Flakes Real Haney", "Kellogs", "KesarPeda_Ajwah", "KesarQueen", "KeshKing",
    "Keya", "KhattaMeetha_FirstCrop", "KhattaMithaMix_Haldirams", "Kimchi", "Kimchi_Samyang",
    "KinderJoy", "Kingfisher", "Kinley", "Kissan", "Kitchen", "KitchenAffairs", "KitchenTreasures",
    "Knorr", "Kurkure", "Kurkure_ChatFills", "Kurkure_SizzlinHot", "Kwality", "Kwality_Muesli",
    "L-oreal", "Lakme", "Lal", "LalMirchPaste_Aram-s", "LaxmanRekhaa", "Layka", "Lays",
    "Lays_American", "Lays_Indian", "Lays_Spanish", "LeCafe", "Lehar", "Lemon_Toothpaste_Colgate",
    "Lifebuoy", "Lime_Dishwash_Prill", "Limonata", "Lindt", "Lipton", "LiquidDetergent_Patanjali",
    "LiquidMosquitoRepellent_GoodNight", "Liril", "Listerine", "Lite", "LittleHearts", "Lizol",
    "LollipopTongueCleaner_DentoShine", "LondonBubble", "LondonMist_BodySpray_YardleyLondon",
    "LondonMist_Perfume_Yardley", "Lotte", "Lotus", "Lux", "M-M", "MB", "MDH",
    "MIxedFruit_Juice_Tropicana", "MTR", "Madhubani", "Madhuri", "Maggi", "Mahakosh", "Makhana",
    "Makino Nacho chips", "Mala_MixedFruitJam", "Malas", "Malkist", "Mangalam", "Mangaldeep",
    "MangoFlavour_ToothpasteForKids_DentoShine", "MangoJuice_Fresca", "MangoMerry",
    "MangoPickle_Tops", "Manna", "Marie_McVities", "Mars", "Marvel", "MarvelTea",
    "MasalaMunch_Kurkure", "MasalaNoodles_TopRamen", "MasalaNoodles_Tops", "MasalaNoodles_maggi",
    "MasalaTikki_Kanwal", "Masala_CuppaNoodles_Maggi", "Masala_TopRamen", "MaxFresh_Toothpaste_Colgate",
    "MaxProtein", "MazedaarMasala_CupNoodles", "Mazic", "McCain", "McVities", "MeatMasala_Orika",
    "Meiji", "MelonSeeds", "MilkBooster_PureBurst", "MilkCompound_Morde", "MilkMagic",
    "MilkShakti_Biscuit_Parle", "Milk_Everyday_Nestle", "Milk_Rusk_Mario", "Milka",
    "Milkshake_Badam_Cavins", "Milkshake_Chocolate_Cavins", "Milkshake_Straberry_Cavins",
    "Milkshake_Vanilla_Cavins", "Milky bar", "MilkyMist", "MiniJumbo_MosquuitoCoil_GoodNight",
    "MiniMeBakers", "MinuteMaid", "MirchiQuormaPaste_Kanwal", "Mishri", "MixedFruitJam_Sil",
    "MixedPickle_Alps", "MixedPickle_Nilon-s", "Mixture_ALLINONE", "Mixture_Ajwah",
    "Mixture_Haldirams", "Mixture_Jabsons", "Mixture_Prabhuji", "Mixture_Rajdhani", "Mixture_Snacko",
    "Mixture_Snactac", "Mixture_Tasty", "Mixture_Tops", "Mixture_Unibic", "Mixture_Vinod",
    "Mixture_Winkies", "Moong_Dal", "MoongDal_Haldirams", "MoongDal_Jabsons", "MoongDal_Prabhuji",
    "MoongDal_Rajdhani", "MoongDal_Snacko", "MoongDal_Snactac", "MoongDal_Tasty", "MoongDal_Tops",
    "MoongDal_Unibic", "MoongDal_Vinod", "MoongDal_Winkies", "Moov", "MotherDairy",
    "MouthFreshner_PanParag", "MouthFreshner_RajniGandha", "Muesli_Bagrrys", "Muesli_Kelloggs",
    "Muesli_Kwality", "Muesli_Saffola", "Muesli_Soulfull", "Munakka_Ajwah", "Munakka_Happilo",
    "Munakka_Nutraj", "Munchies", "Munchkins", "Munchy-s", "Mustard_Oil", "Nandini",
    "NapthaleneBalls_Trishul", "Nataraj", "Natural_Ice_Cream", "NaturalCare_Toothpaste_Colgate",
    "Nescafe", "Nestle", "Nestle_EveryDay", "Neutrogena", "Nido", "Nihar", "Nilon-s", "Nimbark",
    "Nirma", "Nivea", "Noodles_Chings", "Noodles_Knorr", "Noodles_Maggi", "Noodles_Patanjali",
    "Noodles_Ching-s", "Noodles_TopRamen", "Noodles_Yippee", "Nutella", "Nutraj", "Nutties",
    "OatsCookies_Unibic", "OatsRagi_Cookies_Unibic", "Ocean", "Oil_Fortune", "Oil_Gemini",
    "Oil_Saffola", "OliveOil_Borges", "OliveOil_DelMonte", "OliveOil_Leonardo", "Oreo",
    "OrganicIndia", "Orion", "Orios", "Ossum", "Oster", "Ottogi", "Parachute", "Parampara",
    "Paras", "Paras_Ghee", "Parle", "Parle-G", "ParleG", "Parryss", "Patanjali", "Pears",
    "Pediasure", "Pedigree", "Pepsodent", "Perfetti", "Perk", "Pernod", "Persil", "Pesto",
    "PhoolMakhana_Ajwah", "PhoolMakhana_Happilo", "PhoolMakhana_Nutraj", "Pickles_Mother-s",
    "Pickles_Nilon-s", "Pickles_Priya", "Pillsbury", "Pineapple_Juice_Tropicana", "Pista",
    "PistaBadam_Drink", "Plum_Cake", "Pocky", "Ponds", "Popcorn", "Poppins", "Poppins_Candy",
    "Poppins_Chocolate", "Poppins_Toffee", "Pops", "Potato_Chips", "Prabhuji", "Prabhuji_Mixture",
    "Prabhuji_MoongDal", "Prabhuji_Namkeen", "Prabhuji_Snacks", "Pran", "Presto", "Priya",
    "PriyaGold", "Priyagold", "Proteinex", "ProteinPlus",  "PulseCandy", "PureBurst", "PureBurst_Candy", "PureBurst_Chocolate", "PureBurst_Toffee",
    "Quaker", "RCola", "Ragi_Cookies_Unibic", "Rajdhani", "RajniGandha", "Raksha", "Ramdev",
    "RasGulla_Bikano", "RasmalaiMix_Gits", "Rasgulla", "Rasna", "RawMango_Juice_Paper_Boat",
    "ReadyMix_Gits", "Real", "RealActiv", "RealFruit", "RealMix", "RedLabel", "RedLabelTea",
    "Rehmat", "Reliance", "Rexona", "Rin", "RoastedAlmonds_Happilo", "RoastedCashews_Happilo",
    "RoastedPeanuts_Happilo", "RoohAfza", "RoseMilk_Cavins", "Roshan", "Royal", "RoyalChallenge",
    "Ruchi", "Rusk", "Rusk_Britannia", "Rusk_Parle", "Rusks", "Rusks_Premium", "Saffola",
    "Safola", "Sakar", "Sakura", "Samyang", "Santoor", "Saridon", "Savlon", "Schezwan_Chings",
    "Schezwan_Sauce", "Sensodyne", "SetWet", "Shalimar", "Shampoo", "Shampoo_Clinic_Plus",
    "Shampoo_Dove", "Shampoo_Head_Shoulders", "Shampoo_Pantene", "Shampoo_Sunsilk", "Shampoo_Tresemme",
    "Sharbat_Hamdard", "Sharbat_RoohAfza", "Shera", "Shikkakai", "Shimaya", "Shree",
    "ShrimpPickle_Tops", "Signature", "Sil", "Slice", "Snackers", "Snacks", "Snacks_Ajwah",
    "Snacks_Kuch-kuch_Bicano", "Snacks_SnacLite_Haldirams", "Snactac", "Snapin", "Snicker",
    "Snickers", "Snug", "SoYum", "Soap", "Soap_Fresh", "Soap_Liril", "Soap_Neem_Dyna", "Soap_fena",
    "Society", "Sofit", "SoftDrink", "SoftDrink_Rasna", "Softouch", "Soni Fresh",
    "SoyaSticks_Bhujialalji", "SoyaSticks_Jabsons", "Soya_Sauce_Nilons", "Spice_Everest",
    "Spices_Catch", "Spices_FineLife", "Spices_MDH", "Spices_ProNature", "SportingClub_Denver",
    "Spreads_Hershey-s", "SpringBlossom_perfume_YardleyLondon", "SriSri",
    "StarBlossom_Perfume_YardleyLondon", "StarFlowerrazi_Perfume_Yardley", "Stayfree", "Sting",
    "Storia", "StrawberryFlovour_ToothpasteForKids_DentoShine", "Streax",
    "StrongTeeth_Toothpaste_Colgate", "StuffedChilliPickle_Tops", "SubhKamal", "Suffola", "Sugar",
    "SugarFree", "SugarFreeCookies_Gaia", "SugarFree_BiskFarm", "SugarLite", "Suhana",
    "Sultan_Scent_Fogg", "Sundrop", "Sunfeast", "Sunfeast Glucose plus", "SunflowerOil_Cargill",
    "SunflowerSeed_Ajwah", "SunflowerSeeds_King", "Sunrich", "Sunsilk", "SuperSarvottam",
    "SurfExcel", "SurfaceDisinfectantSpray_Savlon", "Svami", "Swach", "Swadist", "Swaminarayan",
    "SweetCorn_Snacko", "SwwetCorn_Sundrop", "SyntheticClearVinegar_Solar", "T-Boost_TrueBasics",
    "TaaliPeanuts", "Taaza", "Tadaa", "TajMahal", "Talati", "Talod", "Tamarind", "Tamarind_Dishwash_Prill",
    "Tamarind_Priya", "Tang", "Tata", "TataGold", "TataPremium", "TataTea taaza", "Tata_Agni",
    "TeaCountry", "TeaTime", "TeaValley", "Tea_ElaichiChai_Shera", "Tea_Mayur", "Tea_Ruby_BrookeBond",
    "Tea_Shera", "Tea_royal", "TeekhaMeetha_Bhujialalji", "TeekhaMeetha_FirstCrop", "Tetley",
    "TheBakersDozen", "TicTac", "Tide", "Tiffany", "Tiger", "Timios", "Tingle", "Toblerone",
    "Toffee_Ajwah", "ToiletCleaner_Expelz", "TomatoChilliSauce_Maggi", "TomatoKetchup",
    "TomatoPuree_GodenCrown", "TomatoPuree_Kissan", "TomatoSoup_Bambino", "TongGarden", "TooYumm",
    "ToothBrush", "ToothPaste", "ToothPaste_CloseUp_ExtremeCool", "ToothPaste_PepsodentG",
    "ToothPowder_Colgate", "Toothbrush_Sensodyne", "ToothpasteForLittleOnes_DentoShine",
    "Toothpaste_Meswak", "Toothpaste_Meswak_Dabur", "Toothpaste_RedGel_Dabur",
    "Toothpaste_Red_BAEFresh_Dabur", "Toothpaste_tulsi_Dabur", "TopBiscuit_Parle", "TopNTown",
    "TopNut", "TopRamen", "Top_Biscuit_Parle", "Top_Ramen", "Tops", "Tops_DrinkingChocolate",
    "TragacanthGum_Ajwah", "Trapa", "Trelish", "Tresemme", "Trident", "TriphalaChurna_Baidyanath",
    "Tropicana", "Truffles_Joyland", "Trump_Scent_Fogg", "TunaNaturalOil_GoldenCrown",
    "TunaOil_GoldenCrown", "TurkishApricot_Happilo", "TurmericPowder_Kanwal", "TurmericPowder_Rehmat",
    "Twinings", "TwistiesNamkeen_FirstCrop", "Twix", "Tycoon_Scent_Fogg", "Ujala",
    "UltraSensual_Deodrant_WildStone", "UncleChips", "Unibic", "Upma_Bambino", "Upma_MTR", "Utsav",
    "VJohn", "VWash", "Vahdam", "VajradantiSF_Toothpaste_Vicco", "Vanish", "Vaseline", "Vatika",
    "Veeba", "VegBiryani_Gits", "VegMayonnaise_Imli_Tree", "VegMayonnaise_Veeba", "VeganMayo_HellManns",
    "Veggi_CupNoodles_Manchow", "VeggieManchow_CupNoodles", "Vermicelli", "Vicks", "Vidal", "Vim",
    "Vim Anti bac", "Vinegar", "Vinegar_Everest", "VisibleWhite_Toothpaste_Colgate",
    "Voyage_Perfume_ParkAvenue", "WaferRoll_Champion", "Wafer_Orange_FullBloom",
    "Wafer_Strawberry_FullBloom", "Wafers_Gastone_Lago", "Wafers_Tirameso_Creweto",
    "Wafers_Treat_Britannia", "WaffleBites_Craveto", "Waffy_Parle", 
    ]
    
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}
    return class_to_idx, class_names

In [5]:
# 4. Data preprocessing functions
def preprocess_dataset(dataset_path, directories, class_to_idx):
    """
    Organize images into proper directory structure
    """
    # Read YOLO format annotations and convert to classification format
    def process_annotations(annotation_file):
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
        # Extract class from first annotation
        if lines:
            class_id = int(lines[0].split()[0])
            return class_id
        return None

    # Process each split (train/valid/test)
    for split in ['train', 'valid', 'test']:
        src_dir = os.path.join(dataset_path, f'{split}/images')
        annotations_dir = os.path.join(dataset_path, f'{split}/labels')
        
        for img_file in os.listdir(src_dir):
            if img_file.endswith(('.jpg', '.jpeg', '.png')):
                # Get corresponding annotation file
                ann_file = os.path.join(annotations_dir, 
                                      img_file.rsplit('.', 1)[0] + '.txt')
                
                if os.path.exists(ann_file):
                    class_id = process_annotations(ann_file)
                    if class_id is not None:
                        # Create class directory if it doesn't exist
                        class_name = class_names[class_id]
                        class_dir = os.path.join(directories[split], class_name)
                        os.makedirs(class_dir, exist_ok=True)
                        
                        # Copy image to appropriate directory
                        shutil.copy2(
                            os.path.join(src_dir, img_file),
                            os.path.join(class_dir, img_file)
                        )

In [6]:
# 5. Define the model architecture
def build_grocery_detection_model(input_shape=(640, 640, 3), num_classes=416):
    """
    Build transfer learning model for grocery detection with EfficientNetB4
    
    Args:
        input_shape: Input image dimensions (640x640 to match dataset)
        num_classes: Number of grocery classes (416)
    """
    # 1. Base Model - EfficientNetB4
    base_model = EfficientNetB4(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = False  # Freeze base model for transfer learning
    
    # 2. Model Architecture
    x = base_model.output
    
    # 3. Global Average Pooling Layer (as per original architecture)
    x = GlobalAveragePooling2D(name='global_average_pooling2d')(x)
    
    # 4. First Dense Block (matching original 1280 neurons)
    x = Dense(1280, name='dense_1')(x)
    x = tf.keras.layers.BatchNormalization(name='bn_1')(x)
    x = tf.keras.layers.ReLU(name='relu_1')(x)
    x = Dropout(0.5, name='dropout_1')(x)  # Increased dropout for regularization
    
    # 5. Second Dense Block (128 neurons as per original)
    x = Dense(128, name='dense_2')(x)
    x = tf.keras.layers.BatchNormalization(name='bn_2')(x)
    x = tf.keras.layers.ReLU(name='relu_2')(x)
    x = Dropout(0.3, name='dropout_2')(x)
    
    # 6. Final Classification Layer with Softmax
    outputs = Dense(num_classes, activation='softmax', name='predictions')(x)
    
    # 7. Create Model
    model = Model(
        inputs=base_model.input, 
        outputs=outputs, 
        name='Grocery_EfficientNetB4'
    )
    
    # 8. Compile Model
    optimizer = Adamax(
        learning_rate=0.001  # As specified in requirements
    )
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',  # As specified
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')
        ]
    )
    
    return model

# Function to print model architecture for verification
def verify_model_architecture(model):
    """
    Verify that all required layers are present
    """
    print("\nModel Architecture Verification:")
    print("================================")
    
    # Required layers
    required_layers = [
        'efficientnetb4',
        'global_average_pooling2d',
        'dense_1',
        'dropout_1',
        'dense_2',
        'dropout_2',
        'predictions'
    ]
    
    # Verify each layer
    found_layers = [layer.name for layer in model.layers]
    print("\nRequired Layers Check:")
    for layer in required_layers:
        status = '✓' if layer in found_layers else '✗'
        print(f"{layer}: {status}")
    
    # Verify optimizer and loss
    print("\nCompilation Settings:")
    print(f"Optimizer: Adamax (lr=0.001)")
    print(f"Loss: categorical_crossentropy")
    
    # Print complete architecture
    print("\nComplete Layer Structure:")
    model.summary()

# Example usage and verification
def create_and_verify_model():
    """
    Create model and verify its architecture
    """
    model = build_grocery_detection_model()
    verify_model_architecture(model)
    return model
  

In [7]:
# 6. Training configuration
def create_data_generators(batch_size=32):
    """
    Create data generators with augmentation
    """
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        zoom_range=0.15,
        shear_range=0.15
    )
    
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    return train_datagen, val_datagen

def setup_callbacks(directories):
    """
    Setup training callbacks
    """
    return [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6
        ),
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(directories['models'], 'best_model.h5'),
            monitor='val_accuracy',
            save_best_only=True
        ),
        tf.keras.callbacks.TensorBoard(
            log_dir=directories['logs']
        )
    ]

In [8]:
# 7. Training function
def train_model(directories, batch_size=32, epochs=50):
    """
    Train the model
    """
    # Create generators
    train_datagen, val_datagen = create_data_generators(batch_size)
    
    # Setup generators
    train_generator = train_datagen.flow_from_directory(
        directories['train'],
        target_size=(640, 640),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    val_generator = val_datagen.flow_from_directory(
        directories['valid'],
        target_size=(640, 640),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    # Build model
    model = build_grocery_detection_model()
    
    # Setup callbacks
    callbacks = setup_callbacks(directories)
    
    # Train model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=callbacks,
        workers=4,
        use_multiprocessing=True
    )
    
    return model, history

In [9]:
# 8. Main execution
if __name__ == "__main__":
    # Setup directories
    directories = setup_directories()
    
    # Create class mapping
    class_to_idx, class_names = create_class_mapping()
    
    # Preprocess dataset
    preprocess_dataset('/kaggle/input/grocery-item-feature-extraction', 
                      directories, class_to_idx)
    
    # Train model
    model, history = train_model(directories)
    
    # Save final model
    model.save(os.path.join(directories['models'], 'final_model.h5'))
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(directories['logs'], 'training_history.png'))
    plt.close()

IndexError: list index out of range